# Experimenting with some pretrained video classification models

In [10]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import numpy as np
import os
from PIL import Image
import torchvision.models.video as video_models
import torch.nn as nn
import tqdm

In [11]:
import video_dataset as Dataset

raw_path = '../data/WLASL2000'
instances_path = './preprocessed_labels/asl100/train_instances_fixed_bboxes_short.json'
classes_path = './wlasl_class_list.json'
train_set = Dataset.VideoDataset(
  root=raw_path,
  instances_path=instances_path,
  classes_path=classes_path,
  transform=Dataset.min_transform_rI3d
)


print(f"Length: {len(train_set)}")

Length: 1442


In [12]:
torch.manual_seed(42)
train_loader = DataLoader(
  train_set,
  batch_size=2,
  shuffle=True,
  num_workers=0 #to use manual seed
)
train_loader

In [13]:
frames, label = next(iter(train_loader))
frames.shape

RuntimeError: Input and output sizes should be greater than 0, but got input (H: 200, W: 0) output (H: 244, W: 244)

In [5]:
#load pretrained mode
model = video_models.r3d_18(pretrained=True)

#alter classifier
num_classes = 100
model.fc = nn.Linear(model.fc.in_features, num_classes)

/home/luke/ExtraStorage/WLASL/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/luke/ExtraStorage/WLASL/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=R3D_18_Weights.KINETICS400_V1`. You can also use `weights=R3D_18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'  worked with cpu
print(f"Device: {device}")

model = model.to(device)

for param in model.parameters():
  param.requires_grad = True

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()
  

Device: cuda


In [7]:
def train_model(model, data_loader, optimizer, loss_func, epochs=10,val_loader=None):
  model.train()
    
  for epoch in tqdm.tqdm(range(epochs), desc="Training R3D"):
    running_loss = 0.0
      
    for batch_idx, (data, target) in enumerate(data_loader):
      # data shape: (batch_size, 3, frames, height, width)
      data, target = data.to(device), target.to(device)
      
      optimizer.zero_grad()
      output = model(data)
      loss = loss_func(output, target)
      loss.backward()
      optimizer.step()
      
      running_loss += loss.item()
      
      # if batch_idx % 100 == 0:
      #     print(f'Epoch [{epoch+1}/{epochs}], Step [{batch_idx}], Loss: {loss.item():.4f}')
      
    print(f'Epoch [{epoch+1}/{epochs}], Average Loss: {running_loss/len(data_loader):.4f}')

In [8]:
train_model(model=model, data_loader=train_loader, optimizer=optimizer, loss_func=loss_func)

Training R3D:   0%|          | 0/10 [00:00<?, ?it/s]


RuntimeError: Input and output sizes should be greater than 0, but got input (H: 223, W: 0) output (H: 244, W: 244)